<a href="https://colab.research.google.com/github/VanThiKimKhanh1203/Hoc_tang_cuong_2024/blob/main/VTKKhanh_bai_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import random
import pandas as pd

In [5]:
# Hàm chuyển trạng thái
def move(state, action):
    x, y = state
    if action == 'Up' and x > 0:
        return x - 1, y
    elif action == 'Down' and x < M - 1:
        return x + 1, y
    elif action == 'Left' and y > 0:
        return x, y - 1
    elif action == 'Right' and y < N - 1:
        return x, y + 1
    return state

# Hàm chọn hành động với độ chính xác 80%
def take_action(action):
    if random.uniform(0, 1) < 0.8:
        return action
    else:
        return random.choice([a for a in actions if a != action])

In [6]:
# Hàm liệt kê tất cả các đường đi khả thi
def find_all_paths(state, path, visited):
    if state == goal:
        all_paths.append(path[:])
        return
    x, y = state
    visited[x][y] = True

    for action in actions:
        next_state = move(state, action)
        nx, ny = next_state
        if not visited[nx][ny] and rewards[nx, ny] != -1:
            path.append((state, action, next_state))
            find_all_paths(next_state, path, visited)
            path.pop()

    visited[x][y] = False

In [7]:
# Nhập dữ liệu từ người dùng
M = int(input("Nhập số hàng M: "))
N = int(input("Nhập số cột N: "))
num_penalty = int(input("Số ô có điểm -1: "))
penalty_cells = []
for _ in range(num_penalty):
    x, y = map(int, input("Nhập tọa độ ô có điểm -1 (dòng, cột): ").split())
    penalty_cells.append((x - 1, y - 1))

goal = tuple(map(int, input("Nhập tọa độ ô có điểm +10 (dòng, cột): ").split()))
goal = (goal[0] - 1, goal[1] - 1)
sub_goal = tuple(map(int, input("Nhập tọa độ ô có điểm +6 (dòng, cột): ").split()))
sub_goal = (sub_goal[0] - 1, sub_goal[1] - 1)
start = tuple(map(int, input("Nhập tọa độ xuất phát của robot (dòng, cột): ").split()))
start = (start[0] - 1, start[1] - 1)

Nhập số hàng M: 3
Nhập số cột N: 4
Số ô có điểm -1: 1
Nhập tọa độ ô có điểm -1 (dòng, cột): 2 2
Nhập tọa độ ô có điểm +10 (dòng, cột): 2 4
Nhập tọa độ ô có điểm +6 (dòng, cột): 2 3
Nhập tọa độ xuất phát của robot (dòng, cột): 3 1


In [8]:
# Thiết lập các thông số của môi trường
actions = ['Up', 'Down', 'Left', 'Right']
rewards = np.zeros((M, N))
rewards[goal] = 10
rewards[sub_goal] = 6
for cell in penalty_cells:
    rewards[cell] = -1

# Thông số Q-learning
Q = np.zeros((M, N, len(actions)))
gamma = 0.9  # Hệ số giảm phần thưởng
alpha = 0.1  # Tốc độ học
epsilon = 0.1  # Tham số khám phá

# Q-learning
for episode in range(1000):
    state = start
    while state != goal:
        # Chọn hành động với epsilon-greedy
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)
        else:
            action = actions[np.argmax(Q[state[0], state[1]])]
        # Áp dụng độ chính xác 80%
        action = take_action(action)
        # Tính trạng thái kế tiếp
        next_state = move(state, action)
        reward = rewards[next_state]
        # Cập nhật Q-value
        Q[state[0], state[1], actions.index(action)] += alpha * (
            reward + gamma * np.max(Q[next_state[0], next_state[1]]) - Q[state[0], state[1], actions.index(action)]
        )
        state = next_state

In [9]:
# Tìm và hiển thị đường đi tối ưu
state = start
optimal_path = []
while state != goal:
    action = actions[np.argmax(Q[state[0], state[1]])]
    action = take_action(action)
    next_state = move(state, action)
    optimal_path.append((state, action, next_state))
    state = next_state

# Hiển thị đường đi tối ưu
print("\nĐường đi tối ưu:")
optimal_path_str = " ; ".join(f"({step[0][0]}, {step[0][1]}) {step[1]} ({step[2][0]}, {step[2][1]})" for step in optimal_path)
print(f"• {optimal_path_str}")
print(f"• {rewards[goal]}")


Đường đi tối ưu:
• (2, 0) Down (2, 0) ; (2, 0) Right (2, 1) ; (2, 1) Right (2, 2) ; (2, 2) Up (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Left (1, 1) ; (1, 1) Right (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Right (0, 3) ; (0, 3) Left (0, 2) ; (0, 2) Right (0, 3) ; (0, 3) Left (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Left (1, 1) ; (1, 1) Right (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Left (0, 1) ; (0, 1) Right (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Up (0, 2) ; (0, 2) Down (1, 2) ; (1, 2) Right (1, 3)
• 10.0


In [10]:
# In bảng Q-value tối ưu
q_table = np.round(Q.max(axis=2), 2)
df_q = pd.DataFrame(q_table)
print("\nBảng Q-value (giá trị tối ưu):")
print(df_q)


Bảng Q-value (giá trị tối ưu):
       0      1      2      3
0  25.58  28.42  31.58  28.42
1  27.42  31.58  28.42   0.00
2  25.58  28.42  31.58  28.42


In [11]:
# Hiển thị bảng hướng di chuyển
action_symbols = {'Up': '^', 'Down': 'v', 'Left': '<', 'Right': '>'}
directions = np.full((M, N), '', dtype=object)
for i in range(M):
    for j in range(N):
        if (i, j) == goal:
            directions[i, j] = '+10'
        elif (i, j) == sub_goal:
            directions[i, j] = '+6'
        elif (i, j) == start:
            directions[i, j] = 'S'
        elif (i, j) in penalty_cells:
            directions[i, j] = '-1'
        else:
            best_action = actions[np.argmax(Q[i, j])]
            directions[i, j] = action_symbols[best_action]
df_directions = pd.DataFrame(directions)
print("\nBảng hướng di chuyển:")
print(df_directions)


Bảng hướng di chuyển:
   0   1   2    3
0  >   >   v    <
1  >  -1  +6  +10
2  S   >   ^    <


In [12]:
# Liệt kê tất cả các đường đi khả thi
visited = [[False for _ in range(N)] for _ in range(M)]
all_paths = []
find_all_paths(start, [], visited)

print("\nTất cả các đường đi khả thi:")
for idx, path in enumerate(all_paths, 1):
    print(f"Đường đi {idx}:")
    for step in path:
        print(f"  ({step[0][0]}, {step[0][1]}) {step[1]} ({step[2][0]}, {step[2][1]})")


Tất cả các đường đi khả thi:
Đường đi 1:
  (2, 0) Up (1, 0)
  (1, 0) Up (0, 0)
  (0, 0) Right (0, 1)
  (0, 1) Right (0, 2)
  (0, 2) Down (1, 2)
  (1, 2) Down (2, 2)
  (2, 2) Right (2, 3)
  (2, 3) Up (1, 3)
Đường đi 2:
  (2, 0) Up (1, 0)
  (1, 0) Up (0, 0)
  (0, 0) Right (0, 1)
  (0, 1) Right (0, 2)
  (0, 2) Down (1, 2)
  (1, 2) Right (1, 3)
Đường đi 3:
  (2, 0) Up (1, 0)
  (1, 0) Up (0, 0)
  (0, 0) Right (0, 1)
  (0, 1) Right (0, 2)
  (0, 2) Right (0, 3)
  (0, 3) Down (1, 3)
Đường đi 4:
  (2, 0) Right (2, 1)
  (2, 1) Right (2, 2)
  (2, 2) Up (1, 2)
  (1, 2) Up (0, 2)
  (0, 2) Right (0, 3)
  (0, 3) Down (1, 3)
Đường đi 5:
  (2, 0) Right (2, 1)
  (2, 1) Right (2, 2)
  (2, 2) Up (1, 2)
  (1, 2) Right (1, 3)
Đường đi 6:
  (2, 0) Right (2, 1)
  (2, 1) Right (2, 2)
  (2, 2) Right (2, 3)
  (2, 3) Up (1, 3)
